In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("penguins.csv")
df.head()

,Unnamed: 0,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [ ]:
del df["Unnamed: 0"]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


In [ ]:
df.species.unique()

array(['Adelie', 'Gentoo', 'Chinstrap'], dtype=object)

In [ ]:
df.island.unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [ ]:
df.year.unique()

array([2007, 2008, 2009])

In [ ]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

In [ ]:
df1 = df.dropna()

In [ ]:
import pandas.api.types as ptp

In [ ]:
for column_name, column_content in df1.items():
    if ptp.is_string_dtype(column_content):
        print(column_name)

species
island
sex


In [ ]:
for column_name, column_content in df1.iteritems():
  if pd.api.types.is_string_dtype(column_content):
    df1[column_name] = column_content.astype("category")

<ipython-input-12-6e4b60cefb40>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[column_name] = column_content.astype("category")


In [ ]:
for column_name, column_content in df1.items():
    if not ptp.is_numeric_dtype(column_content):
        df1[column_name] = df1[column_name].cat.codes

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            333 non-null    int8   
 1   island             333 non-null    int8   
 2   bill_length_mm     333 non-null    float64
 3   bill_depth_mm      333 non-null    float64
 4   flipper_length_mm  333 non-null    float64
 5   body_mass_g        333 non-null    float64
 6   sex                333 non-null    int8   
 7   year               333 non-null    int64  
dtypes: float64(4), int64(1), int8(3)
memory usage: 16.6 KB


<ipython-input-13-38a74a79c9fe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[column_name] = df1[column_name].cat.codes


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df1.drop("species", axis=1)
y = df1.species

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

mu = X_train.mean(axis = 0)
sigma = X_train.std(axis = 0)

X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma
X_train.shape

(266, 7)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor()

In [ ]:
%%time
rf_regressor.fit(X_train, y_train)

CPU times: user 170 ms, sys: 1.57 ms, total: 171 ms
Wall time: 180 ms


RandomForestRegressor()

In [ ]:
rf_regressor.score(X_test, y_test)

0.9804529543994862

In [ ]:
rf_regressor.score(X_train, y_train)

0.9947004400977996

In [ ]:
y_pred = rf_regressor.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score

def rmsle(y_test, y_pred):
  return np.sqrt(mean_squared_log_error(y_test, y_pred))

In [ ]:
rmsle(y_test, y_pred)

0.08565473036424649

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
mdl = keras.Sequential()

mdl.add(layers.Dense(128, activation = "relu", input_shape = (X_train.shape[1],) ))
mdl.add(layers.Dense(64, activation = "relu"))
mdl.add(layers.Dense(3, activation = "linear"))

mdl.compile(optimizer = "adam", loss = "mse", metrics = ["mape"])
mdl.fit(X_train, y_train, epochs=100)

Epoch 1/100
9/9 [==============================] - 1s 4ms/step - loss: 1.6041 - mape: 79721672.0000
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 0.7756 - mape: 128573296.0000
Epoch 3/100
9/9 [==============================] - 0s 3ms/step - loss: 0.3026 - mape: 145459744.0000
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 0.1167 - mape: 94020120.0000
Epoch 5/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0846 - mape: 73636696.0000
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0602 - mape: 68198896.0000
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0477 - mape: 63787900.0000
Epoch 8/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0435 - mape: 62873576.0000
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0396 - mape: 55985516.0000
Epoch 10/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0366 - mape: 51609984.00

In [ ]:
target_column = ['species']
predictors = list(set(list(df1.columns))-set(target_column))
df1[predictors] = df1[predictors]/df1[predictors].max()
df1.describe()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,0.918919,0.325826,0.738134,0.798366,0.869987,0.667787,0.504505,0.999523
std,0.889718,0.357358,0.091756,0.091592,0.060674,0.127812,0.500732,0.000405
min,0.000000,0.000000,0.538591,0.609302,0.744589,0.428571,0.000000,0.999004
25%,0.000000,0.000000,0.662752,0.725581,0.822511,0.563492,0.000000,0.999004
50%,1.000000,0.500000,0.746644,0.804651,0.852814,0.642857,1.000000,0.999502
75%,2.000000,0.500000,0.815436,0.869767,0.922078,0.757937,1.000000,1.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
X = df1[predictors].values
y = df1[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(233, 7)
(100, 7)


In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print(count_classes)

3


In [ ]:
model = keras.Sequential()

model.add(layers.Dense(128, activation='relu', input_dim=7))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
8/8 [==============================] - 1s 3ms/step - loss: 1.0809 - accuracy: 0.4034
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 0.9936 - accuracy: 0.7082
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 0.9439 - accuracy: 0.7768
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: 0.8954 - accuracy: 0.7167
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 0.8459 - accuracy: 0.6695
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 0.7983 - accuracy: 0.6738
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.7472 - accuracy: 0.6652
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.7061 - accuracy: 0.6652
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.6675 - accuracy: 0.6652
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.6384 - accuracy: 0.6996
Epoch 11/100
8/8 [===========

In [ ]:
model.evaluate(X_train, y_train)
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.0233 - accuracy: 1.0000


[0.023259999230504036, 1.0]

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)
y_pred_train = mlp.predict(X_train)


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_score = accuracy_score(y_test, y_pred)
train_score = accuracy_score(y_train, y_pred_train)

In [ ]:
test_score

0.78

In [ ]:
train_score

0.8025751072961373

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3)

In [ ]:
kmeans = kmeans.fit(X_train, y_train)
y_pred_test = kmeans.predict(X_test)
y_pred_train = kmeans.predict(X_train)


In [ ]:
test_score = accuracy_score(y_test, y_pred_test)
train_score = accuracy_score(y_train, y_pred_train)

In [ ]:
test_score

0.14925373134328357

In [ ]:
train_score

0.09022556390977443

In [ ]:
from sklearn import metrics

In [ ]:
metrics.rand_score(y_pred_test, y_pred)

1.0